# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
import requests
import pandas as pd

In [3]:
city_bike_networks = requests.get('http://api.citybik.es/v2/networks')
city_bike_networks

<Response [200]>

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
city_bike_networks = requests.get('http://api.citybik.es/v2/networks').json()
networks = city_bike_networks['networks']
networks_df = pd.json_normalize(data = networks)

networks_df['location.city']

networks_df[(networks_df['location.city'] == 'Vancouver')]["id"]

241    mobibikes
Name: id, dtype: object

In [5]:
#getting json stations data for certain network_id

def network_json(network_id):
    return requests.get("http://api.citybik.es/v2/networks/"+network_id).json()['network']['stations']

#getting  json stations data for vancouver (mobibikes)
    
vancouver_stations= network_json('mobibikes')

vancouver_stations[0]

{'empty_slots': 14,
 'extra': {'ebikes': 4,
  'has_ebikes': True,
  'last_updated': 1676316003,
  'normal_bikes': 17,
  'renting': True,
  'returning': True,
  'slots': 35,
  'uid': '0001'},
 'free_bikes': 21,
 'id': '7a19c49f486d7c0c02b3685d7b240448',
 'latitude': 49.262487,
 'longitude': -123.114397,
 'name': '10th & Cambie',
 'timestamp': '2023-02-13T19:22:31.595000Z'}

Put your parsed results into a DataFrame.

In [16]:
vancouver_stations_df=pd.json_normalize(data=vancouver_stations)

van_bike_data=vancouver_stations_df
van_bike_data['ll'] = van_bike_data['latitude'].astype(str) + ',' + van_bike_data['longitude'].astype(str)
van_bike_data=van_bike_data[['free_bikes','name','ll']]
print(van_bike_data.shape,'\n',van_bike_data.head())

(241, 3) 
    free_bikes                         name                     ll
0          21                10th & Cambie  49.262487,-123.114397
1           9  Yaletown-Roundhouse Station  49.274566,-123.121817
2          13            Dunsmuir & Beatty  49.279764,-123.110154
3           2     12th & Yukon (City Hall)  49.260599,-123.113504
4           9                    8th & Ash  49.264215,-123.117772


In [17]:
# exporting the extracted data as a CSV file 
van_bike_data.to_csv('../data/van_bike_data.csv',index=False)